# Multiclass version of our mushroom classifier

In [157]:
MW_DATASET_VERSION = 'mushroom_world_2017_10_30'
MW_DATASET_URL = 'https://s3.eu-central-1.amazonaws.com/deep-shrooms/{}.zip'.format(MW_DATASET_VERSION)
MW_DATASET_PATH = 'data/{}/'.format(MW_DATASET_VERSION)

LP_DATASET_VERSION = 'luontoportti_2017_10_30'
LP_DATASET_URL = 'https://s3.eu-central-1.amazonaws.com/deep-shrooms/{}.zip'.format(LP_DATASET_VERSION)
LP_DATASET_PATH = 'data/{}/'.format(LP_DATASET_VERSION)

CLASSES = ['edible', 'edible when prepared', 'inedible', 'poisonous', 'deadly']
NUM_CLASSES = len(CLASSES)
INPUT_SHAPE = (256, 256, 3)
BATCH_SIZE = 128
EPOCHS = 12

In [2]:
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

def import_data_set(url):
    with urlopen(url) as zip_response:
        file_size = int(zip_response.headers.get('content-length'))
        print("File {} is {} MB large".format(url, file_size/1000000))
        with ZipFile(BytesIO(zip_response.read())) as zfile:
            zfile.extractall('./data')


In [9]:
import_data_set(MW_DATASET_URL)
import_data_set(LP_DATASET_URL)

File https://s3.eu-central-1.amazonaws.com/deep-shrooms/luontoportti_2017_10_30.zip is 11243985 bytes large


In [10]:
import pandas as pd
import numpy as np

mw_mushroom_classes = pd.read_json(MW_DATASET_PATH + 'mushroom_classes.json', lines=True)
mw_mushroom_imgs = pd.read_json(MW_DATASET_PATH + 'mushroom_imgs.json', lines=True)
mw_mushroom_info = mw_mushroom_imgs.merge(mw_mushroom_classes, how = "right", on = "name_latin")

lp_mushroom_classes = pd.read_csv(LP_DATASET_PATH + 'lp_mushroom_classes.csv', sep='\t')
lp_mushroom_imgs = pd.read_csv(LP_DATASET_PATH + 'lp_mushroom_imgs.csv', sep='\t')

In [8]:
print(lp_mushroom_imgs.at[0, 'edibility'])

syötävyys: o** – keittämisen jälkeen hyvä ruokasieni


## Format labels

Following labels will be formatted into 5 different classes.

### From mushroom.world
* inedible                202
* edible                  149
* poisonous                77
* edible and good          50
* edible and excellent     27
* lethally poisonous       16
* edible when cooked       15

### From luontoportti.fi

* syötävyys: *** – erinomainen ruokasieni 149
* syötävyys: * – syötävä ruokasieni 134
* syötävyys: *** – erinomainen ruokasieni, eviran suosittelema 124
* syötävyys: ** – hyvä ruokasieni 101
* syötävyys: ** – hyvä ruokasieni, eviran suosittelema 72
* syötävyys: o* – keittämisen jälkeen syötävä ruokasieni 51
* syötävyys: x – käyttökelvoton 48
* syötävyys: ††† – tappavan myrkyllinen 39
* syötävyys: o** – keittämisen jälkeen hyvä ruokasieni, eviran suosittelema 36
* syötävyys: † – lievästi myrkyllinen 27
* syötävyys: †† – myrkyllinen 24
* syötävyys: x – käyttökelvoton ruokasienenä 21
* syötävyys: o** – keittämisen jälkeen hyvä ruokasieni 18
* syötävyys: *** – erinomainen ruokasieni, eviran suosittelema. (lajiepiteetti 12
* syötävyys: x – syötäväksi kelpaamaton 12
* syötävyys: o** – keittämisen jälkeen hyvä ruokasieni, ei kuitenkaan suositella syötäväksi useita kertoja viikossa 12
* syötävyys: †††o*** – tuoreena tappavan myrkyllinen, oikein esikäsiteltynä erinomainen ruokasieni, eviran suosittelema 9
* syötävyys: * –  syötävä ruokasieni 9
* syötävyys: o* – ryöppäämisen jälkeen syötävä ruokasieni 9
* syötävyys: ** – hyvä ruokasieni, eviran suosittelema (saa myydä vaaleaorakkaan seassa) 9
* syötävyys: *** –  erinomainen ruokasieni 6
* syötävyys: x – ei syötävä ruokasieni (paitsi maustekäytössä) 5
* syötävyys: ††† –  tappavan myrkyllinen 3

### End result

* edible
* edible when prepared
* inedible
* poisonous
* deadly

In [159]:
mw_edible = ['edible', 'edible and good', 'edible and excellent']
lp_edible = ['syötävyys: *']

mw_edible_when_prepared = ['edible when cooked']
lp_edible_when_prepared = ['syötävyys: o', 'syötävyys: †††o*']

mw_inedible = ['inedible']
lp_inedible = ['syötävyys: x']

mw_poisonous = ['poisonous']
lp_poisonous = ['syötävyys: † –', 'syötävyys: †† –']

mw_deadly = ['lethally poisonous']
lp_deadly = ['syötävyys: †††']

print("Target classes: {}".format(CLASSES))

Target classes: ['edible', 'edible when prepared', 'inedible', 'poisonous', 'deadly']


In [163]:
df2 = mw_mushroom_info.copy(deep=True)
df3 = lp_mushroom_imgs.copy(deep=True)
df2.edibility.value_counts()
#df3.edibility.value_counts()

inedible                202
edible                  149
poisonous                77
edible and good          50
edible and excellent     27
lethally poisonous       16
edible when cooked       15
Name: edibility, dtype: int64

In [161]:
df2.edibility = df2.edibility.where(~df2.edibility.isin(mw_edible), 'edible')
df2.edibility = df2.edibility.where(~df2.edibility.isin(mw_edible_when_prepared), 'edible when prepared')
df2.edibility = df2.edibility.where(~df2.edibility.isin(mw_deadly), 'deadly')

df2.edibility.value_counts()

edible                  226
inedible                202
poisonous                77
deadly                   16
edible when prepared     15
Name: edibility, dtype: int64

In [162]:
df3.edibility = df3.edibility.apply(lambda x: CLASSES[0] if len([s for s in lp_edible if s in x]) > 0 else x)
df3.edibility = df3.edibility.apply(lambda x: CLASSES[1] if len([s for s in lp_edible_when_prepared if s in x]) > 0 else x)
df3.edibility = df3.edibility.apply(lambda x: CLASSES[2] if len([s for s in lp_inedible if s in x]) > 0 else x)
df3.edibility = df3.edibility.apply(lambda x: CLASSES[3] if len([s for s in lp_poisonous if s in x]) > 0 else x)
df3.edibility = df3.edibility.apply(lambda x: CLASSES[4] if len([s for s in lp_deadly if s in x]) > 0 else x)

df3.edibility.value_counts()

edible                  616
edible when prepared    135
inedible                 86
poisonous                51
deadly                   42
Name: edibility, dtype: int64

In [ ]:
a = 'asdf'
b = ['as', 'df']
c = ['cd', 'ef']
x = 'edible' if len([x for x in b if x in a]) > 0 else 'x'
print(x)

In [ ]:
from scipy import misc

def load_mushroom_images(folder_path, img_df):
    img_dict = {}
    for index, path in enumerate(img_df['file_path']):
        img_dict[index] = misc.imread(folder_path + path)
    return img_dict
        
mw_img_dict = load_mushroom_images(MW_DATASET_PATH, mw_mushroom_info)
lp_img_dict = load_mushroom_images(LP_DATASET_PATH, lp_mushroom_img)

# Add dictionaries as new columns to the dataframes
# TODO

In [ ]:
# Merging of the two dataframes together
# TODO

all_mushroom_df = mw_mushroom_info.merge(lp_mushroom_img, how="outer", on="name_latin")

# Drop all useless columns for the training of the model
# Leave only name_latin, edibility img?
# TODO

all_mushroom_img = asdf